<h1 style = "font-size:3rem;color:darkcyan"> Train Classifier</h1>

Using TensorFlow and Keras

In [39]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import json
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [41]:
# import dataset
def import_dataset(dataset_path):
    with open(dataset_path, 'r') as f:
        data = json.load(f)
        
    # split data
    
    # split list into different np arrays
    mappings = np.array(data['mappings']) 
    inputs = np.array(data['MFCC'])
    targets = np.array(data['labels'])
    filenames = np.array(data['filenames'])
    
    return mappings, inputs, targets, filenames

In [53]:
mappings, inputs, targets, filenames = import_dataset('data.json')

In [54]:
# adjusting the input shape for CNN compatibility (from 2D to 3D (excluding # samples))
print('input shape before: {}'.format(inputs.shape))
inputs = inputs[..., np.newaxis] 
print('input shape after: {}'.format(inputs.shape))

input shape before: (95394, 44, 13)
input shape after: (95394, 44, 13, 1)


In [55]:
# splitting data into train, validation, and testing sets
# filenames for misclassification-tracking

def train_validation_test_split(x, y, filenames, test_size = 0.25, val_size = 0.2):
    if (0 < test_size < 1)  and (0 < val_size < 1):
        train_size = 1 - test_size - val_size
        if train_size > 0:

            # train test split
            (
            x_train, x_test, 
            y_train, y_test,
            fn_train, fn_test
            ) = train_test_split(x, y, filenames, test_size = test_size)

            #train validation split
            (
            x_train, x_val, 
            y_train, y_val, 
            fn_train, fn_val
            ) = train_test_split(x_train, y_train, fn_train, test_size = val_size)

            return x_train, x_val, x_test, y_train, y_val, y_test, fn_train, fn_val, fn_test
        else: 
            print('train size is too small, adjust test and validation sizes')
            return
    else:
        print('test and validation size should be between 0 and 1!')
        return

In [57]:
# split data into train, validation and test sets
(
x_train, x_val, x_test,
y_train, y_val, y_test,
fn_train, fn_val, fn_test
) = train_validation_test_split(inputs, targets, filenames, test_size = 0.25, val_size = 0.2)

In [59]:
x_train.shape

(57236, 44, 13, 1)

In [ ]:
DATA_PATH = 'data.json'
LEARNING_RATE = 0.0001

# split data into train, validation, and test
X_train, X_validation, X_test, y_train, y_validation, y_test = get_data_split(DATA_PATH)

# build model
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3] ) #( # segments (sr/hop_size), # n_mfcc, 1)
model = build_model(input_shape, LEARNING_RATE, )

# train model
